In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/train.csv")
test_data = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/test.csv")
test_data.head()

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
feat_with_null = [col for col in train_data.columns if train_data[col].isnull().sum()>10000]
print(feat_with_null)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_data.hist(bins=20,figsize=(20,15))

In [ ]:
sns.histplot(data=train_data,x='Age',hue='Survived',multiple='dodge',shrink=.7,bins=15)

In [ ]:
sns.histplot(data=train_data,x='Sex',hue='Survived',multiple='dodge',shrink=.8)

In [ ]:
sns.histplot(data=train_data,x='Pclass',hue='Survived',multiple='dodge')

In [ ]:
from sklearn.model_selection import train_test_split
target = train_data.Survived
train_data.drop(['Survived','Cabin'],axis=1,inplace=True)
test_data.drop(['Cabin'],axis=1,inplace=True)

too_much_null = [col for col in train_data.columns if train_data[col].isnull().sum()>50000]
train_data.drop(too_much_null,axis=1,inplace=True)
feature_train,feature_valid,label_train,label_valid = train_test_split(train_data,target)
print(feature_train.shape,label_train.shape)

In [ ]:
feature_train.head

In [ ]:
from sklearn.preprocessing import OneHotEncoder

object_cols = [col for col in train_data.columns if train_data[col].dtype == 'object']
cat_cols = [col for col in train_data.columns if train_data[col].dtype == 'object' and train_data[col].nunique()<10]

OHE = OneHotEncoder(handle_unknown='ignore',sparse=False)
OHE_train = pd.DataFrame(OHE.fit_transform(feature_train[cat_cols]))
OHE_valid = pd.DataFrame(OHE.transform(feature_valid[cat_cols]))

OHE_train.index = feature_train.index
OHE_valid.index = feature_valid.index

In [ ]:
nums_cols = [col for col in train_data.columns if train_data[col].dtype in ['int64','float64']]
print(nums_cols)

In [ ]:
print(OHE_train)

In [ ]:
feature_train_num = feature_train.drop(object_cols,axis=1)
feature_valid_num = feature_valid.drop(object_cols,axis=1)
feature_train_clean = pd.concat([feature_train_num,OHE_train],axis=1)
feature_valid_clean = pd.concat([feature_valid_num,OHE_valid],axis=1)

In [ ]:
from xgboost import XGBRegressor
model_1 = XGBRegressor(n_estimators=300,learning_rate=0.05)
model_1.fit(feature_train_clean,label_train)
result = model_1.predict(feature_valid_clean)
print(result)

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(label_valid,result))

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=
                                   [('imputer',SimpleImputer(strategy='most_frequent')),
                                                             ('encoder',OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
('num',numerical_transformer,nums_cols),
('categorical',categorical_transformer,cat_cols)])

In [ ]:
print(cat_cols,feature_train_clean.columns,test_data.columns)

In [ ]:
from xgboost import XGBRegressor
model_1 = XGBRegressor(n_estimators=300,learning_rate=0.05)
pipeline_1 = Pipeline(steps=[('preprocessor',preprocessor),
                            ('model',model_1)])
pipeline_1.fit(feature_train,label_train)
result2 = pipeline_1.predict(feature_valid)

In [ ]:
results = pipeline_1.predict(test_data)
results.shape

In [ ]:
output = test_data.PassengerId.copy()
pred_val = (results>0.5).astype(int)
pred_val = pd.Series(pred_val.tolist(),name='Survived')
#output['Survived'] = pred_val
output = pd.concat([output,pred_val],axis=1)
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
#output['Survived'] = (results>0.5).astype(int).tolist()